# Feedback Plotting

In [1]:
# In this script, we will use the calculated radiative perturbations from various other scripts
# and produce visual representations of Arctic and global feedbacks.

In [2]:
# import statements

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as stats
import xesmf as xe

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [ ]:
# create function for taking spatial averages, while weighting for latitude
def spatial_mean(ds_in, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    zonal = ds_in.mean(dim='lon').sel(lat=slice(lat_bound_s,lat_bound_n))
    weights = np.cos(np.deg2rad(zonal.lat)) / np.sum(np.cos(np.deg2rad(zonal.lat)))
    return((zonal * weights).sum(dim='lat'))

In [ ]:
norm = -1 * plk_g.mean(dim='ens')/dTS_g.mean(dim='ens')

In [ ]:
# get Arctic feedbacks for M1 and 2Y

plk_a_norm = ((plk_a/dTS_a.mean(dim='ens') - plk_g/dTS_g.mean(dim='ens')) * dTS_a.mean(dim='ens') / norm)
plk_a_std = ((plk_a/dTS_a.mean(dim='ens') - plk_g/dTS_g.mean(dim='ens')) * dTS_a.mean(dim='ens') / norm).std(dim='ens')
plk_ps = stats.ttest_1samp(((plk_a/dTS_a.mean(dim='ens') - plk_g/dTS_g.mean(dim='ens')) * dTS_a.mean(dim='ens') / norm),0,axis=0)[1]
plk_g_norm = -1 * ((plk_g / dTS_g.mean(dim='ens')) - (plk_g / dTS_g.mean(dim='ens'))) * dTS_g / norm

LR_AM1 = (lr_a/norm).isel(time=slice(None,31))
PL_AM1 = (plk_a_norm).isel(time=slice(None,31))
Q_AM1 = ((qlw_a+qsw_a)/norm).isel(time=slice(None,31))
Alb_AM1 = (alb_a/norm).isel(time=slice(None,31))
Cld_AM1 = ((cloudlw_a + cloudsw_a)/norm).isel(time=slice(None,31))
OHU_AM1 = ((fsfc_a/norm)).isel(time=slice(None,31))
AHT_AM1 = ((AHT/norm)).isel(time=slice(None,31))

LR_AY2 = lr_a/norm
PL_AY2 = plk_a_norm
Q_AY2 = ((qlw_a+qsw_a)/norm)
Alb_AY2 = alb_a/norm
Cld_AY2 = (cloudlw_a + cloudsw_a)/norm
OHU_AY2 = fsfc_a/norm
AHT_AY2 = AHT/norm

LR_GM1 = (lr_g/norm).isel(time=slice(None,31))
PL_GM1 = 0
Q_GM1 = ((qlw_g+qsw_g)/norm).isel(time=slice(None,31))
Alb_GM1 = (alb_g/norm).isel(time=slice(None,31))
Cld_GM1 = ((cloudlw_g + cloudsw_g)/norm).isel(time=slice(None,31))
OHU_GM1 = (fsfc_g/norm).isel(time=slice(None,31))
AHT_GM1 = 0

LR_GY2 = lr_g/norm
PL_GY2 = 0
Q_GY2 = ((qlw_g + qsw_g)/norm)
Alb_GY2 = alb_g/norm
Cld_GY2 = (cloudlw_g + cloudsw_g)/norm
OHU_GY2 = fsfc_g/norm
AHT_GY2 = 0

In [ ]:
arc_means = [LR_AM1.mean(dim=['time','ens']),PL_AM1.mean(dim=['time','ens']),Q_AM1.mean(dim=['time','ens']),Alb_AM1.mean(dim=['time','ens']),Cld_AM1.mean(dim=['time','ens']),
            OHU_AM1.mean(dim=['time','ens'])]
glb_means = [LR_GM1.mean(dim=['time','ens']),PL_GM1,Q_GM1.mean(dim=['time','ens']),Alb_GM1.mean(dim=['time','ens']),Cld_GM1.mean(dim=['time','ens']),
            OHU_GM1.mean(dim=['time','ens'])]
        
x = np.arange(len(arc_means))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(7,3.5))
rects1 = ax.bar(x - width/2, arc_means, width, label='Arctic',color='red')
rects1a = ax.bar(x[-1]+1 - width/2,AHT_AM1.mean(dim=['time','ens']),width,color='red',hatch='///')
rects1b = ax.bar(x[-1]+2 - width/2,spatial_mean(ERF.dF_as,lat_bound_s=70)/norm.mean(dim='time'),width,color='red')
rects2 = ax.bar(x + width/2, glb_means, width, label='global',color='blue')
rects2a = ax.bar(x[-1]+1 + width/2,AHT_GM1,width,color='blue',hatch='//')
rects2b = ax.bar(x[-1]+2 + width/2,spatial_mean(ERF.dF_as)/norm.mean(dim='time'),width,color='blue')

plt.text(x[0]-width/2,-0.2,'LR')
plt.text(x[1]-width/2,-0.2,'P\'')
plt.text(x[2]-width/2,-0.2,'WV')
plt.text(x[3]-width/2,-0.2,'Alb')
plt.text(x[4]-width/2,0.1,'Cld')
plt.text(x[5]-width/2,0.1,'OHU')
plt.text(x[5]+1-width/2,-0.2,'AHT')
plt.text(x[5]+2-width/2,-0.2,'ERF')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Warming Contribution (K)')
ax.set_title('First month after 4xCO$_2$')
ax.set_xticks(x)
ax.spines["bottom"].set_position(("data", 0))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticks([])
# ax.set_xticklabels(labels)
ax.legend()
ax.grid()
ax.set_ylim([-2.3,2.3])

fig.tight_layout()


plt.savefig('feedbacks_1M_bar_chart.svg')

In [ ]:
arc_means = [LR_AY2.mean(dim=['time','ens']),PL_AY2.mean(dim=['time','ens']),Q_AY2.mean(dim=['time','ens']),Alb_AY2.mean(dim=['time','ens']),Cld_AY2.mean(dim=['time','ens']),
            OHU_AY2.mean(dim=['time','ens'])]
glb_means = [LR_GY2.mean(dim=['time','ens']),PL_GY2,Q_GY2.mean(dim=['time','ens']),Alb_GY2.mean(dim=['time','ens']),Cld_GY2.mean(dim=['time','ens']),
            OHU_GY2.mean(dim=['time','ens'])]
        
x = np.arange(len(arc_means))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(7,3.5))
rects1 = ax.bar(x - width/2, arc_means, width, label='Arctic',color='red')
rects1a = ax.bar(x[-1]+1 - width/2,AHT_AY2.mean(dim=['time','ens']),width,color='red')
rects1b = ax.bar(x[-1]+2 - width/2,spatial_mean(ERF.dF_as,lat_bound_s=70)/norm.mean(dim='time'),width,color='red')
rects2 = ax.bar(x + width/2, glb_means, width, label='global',color='blue')
rects2a = ax.bar(x[-1]+1 + width/2,AHT_GY2,width,color='blue',hatch='//')
rects2b = ax.bar(x[-1]+2 + width/2,spatial_mean(ERF.dF_as)/norm.mean(dim='time'),width,color='blue')

plt.text(x[0]-width/2,-0.5,'LR')
plt.text(x[1]-width/2,-0.2,'P\'')
plt.text(x[2]-width/2,-0.2,'WV')
plt.text(x[3]-width/2,-0.2,'Alb')
plt.text(x[4]-width/2,0.1,'Cld')
plt.text(x[5]-width/2,0.1,'OHU')
plt.text(x[5]+1-width/2,-0.2,'AHT')
plt.text(x[5]+2-width/2,-0.2,'ERF')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Warming Contribution (K)')
ax.set_title('First 2 years after 4xCO$_2$')
ax.set_xticks(x)
ax.spines["bottom"].set_position(("data", 0))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticks([])
# ax.set_xticklabels(labels)
# ax.legend()
ax.set_ylim([-2.3,2.3])
ax.grid()

fig.tight_layout()

plt.savefig('feedbacks_2Y_bar_chart.svg')

In [ ]:
# create figure object
fig, ax = plt.subplots(ncols=1,figsize=(5,4))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_diff/2, color='red',label='Alb/2')
ax.plot(xs, q_diff, color='indigo',label='WV')
ax.plot(xs, lr_diff, color='green',label='LR')
ax.plot(xs, plk_a_norm, color='darkorange',label='Planck')
# ax[2,0].set_ylim([-3,14])
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=8)
ax.tick_params(axis='y',labelsize=8)
ax.set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
# ax[2,0].set_title(r'Change in SAT, 4xCO$_2$ in Jan, 30 ens. members',fontsize=10)
ax.legend(loc='upper left',fontsize=8)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

plt.title('TOA perspective')
plt.tight_layout()
plt.show()

In [ ]:
# create figure object
fig, ax = plt.subplots(nrows=2,figsize=(8,5),sharex=True,sharey=True)

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax[0].plot(xs, lr_diff, color='green',label='LR',linewidth=2)
ax[0].fill_between(xs, lr_diff - lr_std, lr_diff + lr_std,color='green',alpha=0.4)
ax[1].plot(xs, plk_a_norm, color='darkorange',label='Planck',linewidth=2)
ax[1].fill_between(xs, plk_a_norm - plk_a_std, plk_a_norm + plk_a_std,color='darkorange',alpha=0.4)
ax[1].set_xlim([1,731])

ax[1].tick_params(axis='x', rotation=45,labelsize=10)
ax[1].tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax[1].hlines(0,1,731,color='black',linestyle=':')
ax[0].hlines(0,1,731,color='black',linestyle=':')
ax[0].legend(loc='upper left',fontsize=10)
ax[1].legend(loc='upper left',fontsize=10)
ax[1].set_xticks(x_vals)
ax[1].set_xticklabels(x_labs)
ax[1].grid()
ax[0].grid()

ax[0].set_ylabel('Diff in warming contr. (K)')
ax[1].set_ylabel('Diff in warming contr. (K)')
ax[0].set_ylim([-5,5])

plt.tight_layout()
plt.savefig('temp_feedbacks.svg')

In [ ]:
# create figure object
fig, ax = plt.subplots(figsize=(8,3))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, q_diff, color='indigo',label='Water vapor',linewidth=2)
ax.fill_between(xs, q_diff - q_std, q_diff + q_std,color='indigo',alpha=0.4)
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=10)
ax.tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
ax.legend(loc='upper left',fontsize=10)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

ax.set_ylabel('Diff in warming contr. (K)')

plt.tight_layout()
ax.set_ylim([-5,5])
# plt.show()
plt.savefig('wv_feedbacks.svg')

In [ ]:
# create figure object
fig, ax = plt.subplots(figsize=(8,3))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, alb_diff, color='red',label='α',linewidth=2)
ax.fill_between(xs, alb_diff - alb_std, alb_diff + alb_std,color='red',alpha=0.4)
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=10)
ax.tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
ax.legend(loc='upper left',fontsize=10)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()
ax.set_ylim([-12,12])

ax.set_ylabel('Diff in warming contr. (K)')

plt.tight_layout()
# plt.show()
plt.savefig('alb_feedbacks.svg')

In [ ]:
# create figure object
fig, ax = plt.subplots(figsize=(8,3))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs, fsfc_diff, color='blue',label='OHU',linewidth=2)
ax.fill_between(xs, fsfc_diff - fsfc_std, fsfc_diff + fsfc_std,color='blue',alpha=0.4)
ax.set_xlim([1,731])

ax.tick_params(axis='x', rotation=45,labelsize=10)
ax.tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
ax.legend(loc='upper left',fontsize=10)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()

ax.set_ylabel('Diff in warming contr. (K)')
ax.set_ylim([-12,12])

plt.tight_layout()
plt.savefig('OHU_diff.svg')

In [ ]:
# create figure object
fig, ax = plt.subplots(figsize=(8,3))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs,lr_diff.where(lr_ps<0.05),color='green',label='LR',linewidth=1.5)
ax.plot(xs,lr_diff,color='green',linewidth=1.5,linestyle='dotted')
ax.plot(xs,plk_a_norm.where(plk_ps<0.05),color='goldenrod',label='P\'',linewidth=1.5)
ax.plot(xs,plk_a_norm,color='goldenrod',linewidth=1.5,linestyle='dotted')
ax.plot(xs,q_diff.where(q_ps<0.05),color='darkorchid',label='Q',linewidth=1.5)
ax.plot(xs,q_diff,color='darkorchid',linewidth=1.5,linestyle='dotted')
ax.plot(xs,alb_diff.where(alb_ps<0.05),color='red',label='Alb',linewidth=1.5)
ax.plot(xs,alb_diff,color='red',linewidth=1.5,linestyle='dotted')
ax.plot(xs,cld_diff.where(cld_ps<0.05),color='dodgerblue',label='Cld',linewidth=1.5)
ax.plot(xs,cld_diff,color='dodgerblue',linewidth=1.5,linestyle='dotted')
ax.plot(xs,fsfc_diff.where(fsfc_ps<0.05),color='navy',label='OHU',linewidth=1.5)
ax.plot(xs,fsfc_diff,color='navy',linewidth=1.5,linestyle='dotted')
# ax.plot(xs,AHT_diff.where(AHT_ps<0.05),color='fuchsia',label='AHT',linewidth=1.5)
# ax.plot(xs,AHT_diff,color='fuchsia',linewidth=1.5,linestyle='dotted')
# ax.plot(xs, fsfc_diff, color='blue',label='OHU',linewidth=2)
ax.set_xlim([1,731])
ax.hlines(spatial_mean(ERF.dF_as,lat_bound_s=70)/norm.mean(dim='time') - spatial_mean(ERF.dF_as)/norm.mean(dim='time'),
          1,731,color='gray',linestyle='dashdot',label='ERF')

ax.tick_params(axis='x', rotation=45,labelsize=10)
ax.tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
ax.legend(loc='lower center',fontsize=9,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()
ax.set_ylim([-10,10])

ax.set_ylabel('Diff in warming contr. (K)',fontsize=10)

plt.tight_layout()
# plt.show()
plt.savefig('2Y_feedbacks_vs_time.svg')

In [ ]:
# create figure object
fig, ax = plt.subplots(figsize=(8,3))

xs = np.arange(1,92,1)

first_days = np.array([1,32,60,91])

x_vals = first_days
x_labs = ['Jan 1','Feb 1','Mar 1','Apr 1']

# plot 4xCO2 starting in Jan
ax.plot(xs,lr_diff.where(lr_ps<0.05).isel(time=slice(None,91)),color='green',label='LR',linewidth=1.5)
ax.plot(xs,lr_diff.isel(time=slice(None,91)),color='green',linewidth=1.5,linestyle='dotted')
ax.plot(xs,plk_a_norm.where(plk_ps<0.05).isel(time=slice(None,91)),color='goldenrod',label='P\'',linewidth=1.5)
ax.plot(xs,plk_a_norm.isel(time=slice(None,91)),color='goldenrod',linewidth=1.5,linestyle='dotted')
ax.plot(xs,q_diff.where(q_ps<0.05).isel(time=slice(None,91)),color='darkorchid',label='Q',linewidth=1.5)
ax.plot(xs,q_diff.isel(time=slice(None,91)),color='darkorchid',linewidth=1.5,linestyle='dotted')
ax.plot(xs,alb_diff.where(alb_ps<0.05).isel(time=slice(None,91)),color='red',label='Alb',linewidth=1.5)
ax.plot(xs,alb_diff.isel(time=slice(None,91)),color='red',linewidth=1.5,linestyle='dotted')
ax.plot(xs,cld_diff.where(cld_ps<0.05).isel(time=slice(None,91)),color='dodgerblue',label='Cld',linewidth=1.5)
ax.plot(xs,cld_diff.isel(time=slice(None,91)),color='dodgerblue',linewidth=1.5,linestyle='dotted')
ax.plot(xs,fsfc_diff.where(fsfc_ps<0.05).isel(time=slice(None,91)),color='navy',label='OHU',linewidth=1.5)
ax.plot(xs,fsfc_diff.isel(time=slice(None,91)),color='navy',linewidth=1.5,linestyle='dotted')
# ax.plot(xs,AHT_diff.where(AHT_ps<0.05),color='fuchsia',label='AHT',linewidth=1.5)
# ax.plot(xs,AHT_diff,color='fuchsia',linewidth=1.5,linestyle='dotted')
# ax.plot(xs, fsfc_diff, color='blue',label='OHU',linewidth=2)

ax.tick_params(axis='x', rotation=45,labelsize=10)
ax.tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle='solid')
ax.hlines(spatial_mean(ERF.dF_as,lat_bound_s=70)/norm.mean(dim='time') - spatial_mean(ERF.dF_as)/norm.mean(dim='time'),
          1,731,color='gray',linestyle='dashdot',label='ERF')
# ax.legend(loc='lower right',fontsize=10)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()
ax.set_ylim([-2.3,2.3])
plt.xlim([1,91])

ax.set_ylabel('Diff in warming contr. (K)',fontsize=10)

plt.tight_layout()
# plt.show()
plt.savefig('3M_feedbacks_vs_time.svg')

In [ ]:
# get Arctic feedbacks for M1 and 2Y

LR_AM1 = (lr_a/norm).isel(time=slice(None,31))
PL_AM1 = (plk_a_norm).isel(time=slice(None,31))
Q_AM1 = ((qlw_a+qsw_a)/norm).isel(time=slice(None,31))
Alb_AM1 = (alb_a/norm).isel(time=slice(None,31))
Cld_AM1 = ((cloudlw_a + cloudsw_a)/norm).isel(time=slice(None,31))
OHU_AM1 = ((fsfc_a/norm)).isel(time=slice(None,31))
AHT_AM1 = ((AHT/norm)).isel(time=slice(None,31))

LR_AY2 = lr_a/norm
PL_AY2 = plk_a_norm
Q_AY2 = ((qlw_a+qsw_a)/norm)
Alb_AY2 = alb_a/norm
Cld_AY2 = (cloudlw_a + cloudsw_a)/norm
OHU_AY2 = fsfc_a/norm
AHT_AY2 = AHT/norm

LR_GM1 = (lr_g/norm).isel(time=slice(None,31))
PL_GM1 = 0
Q_GM1 = ((qlw_g+qsw_g)/norm).isel(time=slice(None,31))
Alb_GM1 = (alb_g/norm).isel(time=slice(None,31))
Cld_GM1 = ((cloudlw_g + cloudsw_g)/norm).isel(time=slice(None,31))
OHU_GM1 = (fsfc_g/norm).isel(time=slice(None,31))
AHT_GM1 = 0

LR_GY2 = lr_g/norm
PL_GY2 = 0
Q_GY2 = ((qlw_g + qsw_g)/norm)
Alb_GY2 = alb_g/norm
Cld_GY2 = (cloudlw_g + cloudsw_g)/norm
OHU_GY2 = fsfc_g/norm
AHT_GY2 = 0

In [ ]:
ERF

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.xaxis.set_tick_params(bottom=True, top=False)
ax.yaxis.set_tick_params(left=True, right=False)
ax.spines['bottom'].set_position('zero') # spine for xaxis 
ax.spines['left'].set_position('zero')  # spine for yaxis 
ax.tick_params(axis='both',labelsize=8)
ax.grid()

ax.scatter(PL_GM1,
                  PL_AM1.mean(dim=['time','ens']),label='P\'',color='darkorange',s=75)
ax.scatter(LR_GM1.mean(dim=['time','ens']),
                  LR_AM1.mean(dim=['time','ens']),color='green',label='LR',s=75)
ax.scatter(Alb_GM1.mean(dim=['time','ens']),
                  Alb_AM1.mean(dim=['time','ens']),color='red',label='Albedo',s=75)
ax.scatter(Q_GM1.mean(dim=['time','ens']),
                  Q_AM1.mean(dim=['time','ens']),color='purple',label='WV',s=75)
ax.scatter(Cld_GM1.mean(dim=['time','ens']),
                  Cld_AM1.mean(dim=['time','ens']),color='turquoise', label='Cloud',s=75)
ax.scatter(AHT_GM1,
                  AHT_AM1.mean(dim=['time','ens']),color='orchid',label='AHT',s=75,facecolors='none')
ax.scatter(OHU_GM1.mean(dim=['time','ens']),
                  OHU_AM1.mean(dim=['time','ens']),color='Navy',label='OHU',linewidth=1.5,s=75)
ax.scatter(spatial_mean(ERF.dF_as)/norm.mean(dim='time'),
                  spatial_mean(ERF.dF_as,lat_bound_s=70)/norm.mean(dim='time'),color='gray',label='ERF',linewidth=1.5,s=75)
ax.set_xlim([-2.4,2.4])
# ax.set_ylim([-1,1])
# ax.set_xticks(np.arange(-2,2.6,0.5))
ax.plot(np.arange(-2.5,2.5,0.5),np.arange(-2.5,2.5,0.5),color='black',linestyle='--')

# ######################################################################################################################
# # Let's see if we can add text labels to this
# ax.text(-2,4,'Arctic (K)',fontsize=8, rotation=90)

# ax.text(3,-1.9,'Global (K)', fontsize=8)
# ax.set_title('Last 30 years',fontsize=8)

# ax.text(5.33,6.22,'+AA',fontsize=7,rotation=46)
# ax.text(6.4,5.511,'-AA',fontsize=7,rotation=46)

plt.tight_layout()
plt.show()
# plt.savefig('feedback_scatter_CAM3_albedo_060820.pdf')

In [ ]:
t,p = stats.ttest_1samp((PL_AM1).mean(dim=['time']),0,axis=0)

In [ ]:
PL_AM1.mean(dim='time')

In [ ]:
plk_a_norm

## TOA plots

In [3]:
gw = xr.open_dataarray('/dx02/janoski/cesm-LE/output/cam5_gauss_weights.nc')
# create function for taking spatial averages, while weighting for latitude
def gw_mean(ds_in, gw, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of, ALREADY ZONALLY AVERAGED
    gw -- Array of guassian weights. Should only have latitude dimension.
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    return (ds_in.sel(lat=slice(lat_bound_s,lat_bound_n)) * gw.sel(lat=slice(lat_bound_s,lat_bound_n))/gw.sel(
        lat=slice(lat_bound_s,lat_bound_n)).sum(dim='lat')).sum(dim='lat')

In [4]:
# read in data file
fin = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM4_CAMRT_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(fin, ds_out, 'bilinear',periodic=True)

# get difference b/w clear-sky and all-sky fluxes
# note: longwave has to be multiplied by -1 for directional consistency
dRFLW_TOA = -1 * (regridder(fin.FLNT))
dRFSW_TOA = regridder(fin.FSNTC) - regridder(fin.FSNT)
dRFLW_SFC = -1 *  regridder(fin.FLNS)
dRFSW_SFC = regridder(fin.FSNS)

/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/xesmf/frontend.py:478: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


In [ ]:
dTS = []
plk = []
lr = []
qlw = []
qsw = []
fsfc = []
alb = []
aht = []
cloudsw = []
cloudlw = []
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_dTS.nc'
    fin = xr.open_dataarray(path,use_cftime=True)
    fin['time'] = np.arange(0,730,1)
    dTS.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Planck_rad_perturbs_updated.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNT']
    fin['time'] = np.arange(0,730,1)
    plk.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_lapse_rad_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNT']
    fin['time'] = np.arange(0,730,1)
    lr.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Q_rad_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNT']
    fin['time'] = np.arange(0,730,1)
    qlw.append(fin.mean(dim='lon'))
    fin = xr.open_dataset(path,use_cftime=True)['FSNT']
    fin['time'] = np.arange(0,730,1)
    qsw.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Fsfc_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['dFsfc']
    fin['time'] = np.arange(0,730,1)
    fsfc.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_alb_rad_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FSNT']
    fin['time'] = np.arange(0,730,1)
    alb.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_AHT_70N.nc'
    fin = xr.open_dataset(path,use_cftime=True)['MSE']
    fin['time'] = np.arange(0,730,1)
    aht.append(fin)
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM4.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_SW']
    fin['time'] = np.arange(0,730,1)
    cloudsw.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM4.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_LW']
    fin['time'] = np.arange(0,730,1)
    cloudlw.append(fin.mean(dim='lon'))
    
dTS = xr.concat(dTS,dim='ens')
dTS['ens'] = np.arange(1,101,1)
plk = xr.concat(plk,dim='ens')
plk['ens'] = np.arange(1,101,1)
lr = xr.concat(lr,dim='ens')
lr['ens'] = np.arange(1,101,1)
qlw = xr.concat(qlw,dim='ens')
qlw['ens'] = np.arange(1,101,1)
qsw = xr.concat(qsw,dim='ens')
qsw['ens'] = np.arange(1,101,1)
fsfc = xr.concat(fsfc,dim='ens')
fsfc['ens'] = np.arange(1,101,1)
alb = xr.concat(alb,dim='ens')
alb['ens'] = np.arange(1,101,1)
aht = xr.concat(aht,dim='ens')
aht['ens'] = np.arange(1,101,1)
cloudsw = xr.concat(cloudsw,dim='ens')
cloudsw['ens'] = np.arange(1,101,1)
cloudlw = xr.concat(cloudlw,dim='ens')
cloudlw['ens'] = np.arange(1,101,1)

In [ ]:
# to convert to temperature contributions, we divide by global average Planck feedback
norm = -1 * gw_mean(plk,gw).mean(dim='ens') / gw_mean(dTS,gw).mean(dim='ens')

norm.plot()

In [ ]:
# get the differences between Arctic and global warming contributions
# variables with diff will already have the ensemble mean taken
lr_diff = ((gw_mean(lr,gw,lat_bound_s=70) - gw_mean(lr,gw))/norm).mean(dim='ens')
lr_ps = stats.ttest_1samp(((gw_mean(lr,gw,lat_bound_s=70) - gw_mean(lr,gw))/norm),0,axis=0)[1]
lr_diff.where(lr_ps<0.05).plot()

# planck is a little bit different - we consider the deviation of the local planck from the global mean
plk_diff = (-1 * (gw_mean(plk,gw) / gw_mean(dTS,gw).mean(dim='ens')) + (gw_mean(plk,gw,lat_bound_s=70) /
                                                        gw_mean(dTS,gw,lat_bound_s=70).mean(dim='ens')))
plk_diff = (plk_diff * gw_mean(dTS,gw,lat_bound_s=70).mean(dim='ens') / norm)
plk_ps = stats.ttest_1samp(plk_diff,0,axis=0)[1]
plk_diff = plk_diff.mean(dim='ens')
plk_diff.where(plk_ps<0.05).plot()

# water vapor
q_diff = ((gw_mean(qsw + qlw,gw,lat_bound_s=70) - gw_mean(qsw+qlw,gw))/norm).mean(dim='ens')
q_ps = stats.ttest_1samp(((gw_mean(qsw+qlw,gw,lat_bound_s=70) - gw_mean(qsw+qlw,gw))/norm),0,axis=0)[1]
q_diff.where(q_ps<0.05).plot()

# albedo
alb_diff = ((gw_mean(alb,gw,lat_bound_s=70) - gw_mean(alb,gw))/norm).mean(dim='ens')
alb_ps = stats.ttest_1samp(((gw_mean(alb,gw,lat_bound_s=70) - gw_mean(alb,gw))/norm),0,axis=0)[1]
alb_diff.where(alb_ps<0.05).plot()

# cloud
cld_diff = ((gw_mean(cloudsw + cloudlw,gw,lat_bound_s=70) - gw_mean(cloudsw+cloudlw,gw))/norm).mean(dim='ens')
cld_ps = stats.ttest_1samp(((gw_mean(cloudsw+cloudlw,gw,lat_bound_s=70) - gw_mean(cloudsw+cloudlw,gw))/norm),0,axis=0)[1]
cld_diff.where(cld_ps<0.05).plot()

# aht
aht_diff = (aht/0.15e14/norm).mean(dim='ens')
aht_ps = stats.ttest_1samp(aht/0.15e14/norm,0,axis=0)[1]

# fsfc
fsfc_diff = ((gw_mean(fsfc,gw,lat_bound_s=70) - gw_mean(fsfc,gw))/norm).mean(dim='ens')
fsfc_ps = stats.ttest_1samp(((gw_mean(fsfc,gw,lat_bound_s=70) - gw_mean(fsfc,gw))/norm),0,axis=0)[1]
fsfc_diff.where(fsfc_ps<0.05).plot()

#co2
co2_diff = ((np.tile(gw_mean((dRFLW_TOA + dRFSW_TOA).mean(dim='lon'),gw,lat_bound_s=70),(2)) - np.tile(gw_mean((dRFLW_TOA + dRFSW_TOA).mean(dim='lon')
                                                                                                               ,gw),(2)))/norm)
plt.plot(np.arange(1,731,1),co2_diff)

In [ ]:
# create figure object
fig, ax = plt.subplots(figsize=(8,3))

xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

# plot 4xCO2 starting in Jan
ax.plot(xs,lr_diff.where(lr_ps<0.05),color='green',label='LR',linewidth=1.5)
ax.plot(xs,lr_diff,color='green',linewidth=1.5,linestyle='dotted')
ax.plot(xs,plk_diff.where(plk_ps<0.05),color='goldenrod',label='P\'',linewidth=1.5)
ax.plot(xs,plk_diff,color='goldenrod',linewidth=1.5,linestyle='dotted')
ax.plot(xs,q_diff.where(q_ps<0.05),color='darkorchid',label='Q',linewidth=1.5)
ax.plot(xs,q_diff,color='darkorchid',linewidth=1.5,linestyle='dotted')
ax.plot(xs,alb_diff.where(alb_ps<0.05),color='red',label='Alb',linewidth=1.5)
ax.plot(xs,alb_diff,color='red',linewidth=1.5,linestyle='dotted')
ax.plot(xs,cld_diff.where(cld_ps<0.05),color='dodgerblue',label='Cld',linewidth=1.5)
ax.plot(xs,cld_diff,color='dodgerblue',linewidth=1.5,linestyle='dotted')
ax.plot(xs,fsfc_diff.where(fsfc_ps<0.05),color='navy',label='OHU',linewidth=1.5)
ax.plot(xs,fsfc_diff,color='navy',linewidth=1.5,linestyle='dotted')
ax.set_xlim([1,731])
ax.plot(xs,co2_diff,color='gray',linewidth=1.5,linestyle='solid',label='CO2')

ax.tick_params(axis='x', rotation=45,labelsize=10)
ax.tick_params(axis='y',labelsize=10)
# ax[1].set_ylabel(r'Diff in warming contr. (K)',fontsize=10)
ax.hlines(0,1,731,color='black',linestyle=':')
ax.legend(loc='lower center',fontsize=9,ncol=3)
ax.set_xticks(x_vals)
ax.set_xticklabels(x_labs)
ax.grid()
ax.set_ylim([-11,11])

ax.set_ylabel('Diff in warming contr. (K)',fontsize=10)

plt.tight_layout()
plt.show()
# plt.savefig('2Y_feedbacks_vs_time.svg')

## sfc plots

In [5]:
dTS = []
Tatm = []
Tsfc = []
qlw = []
qsw = []
fsfc = []
alb = []
aht = []
cloudsw = []
cloudlw = []
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_dTS.nc'
    fin = xr.open_dataarray(path,use_cftime=True)
    fin['time'] = np.arange(0,730,1)
    dTS.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Ts_rad_perturbs_sfc.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNS']
    fin['time'] = np.arange(0,730,1)
    Tsfc.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Ta_rad_perturbs_sfc.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNS']
    fin['time'] = np.arange(0,730,1)
    Tatm.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Q_rad_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNS']
    fin['time'] = np.arange(0,730,1)
    qlw.append(fin.mean(dim='lon'))
    fin = xr.open_dataset(path,use_cftime=True)['FSNS']
    fin['time'] = np.arange(0,730,1)
    qsw.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Fsfc_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['dFsfc']
    fin['time'] = np.arange(0,730,1)
    fsfc.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_alb_rad_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FSNS']
    fin['time'] = np.arange(0,730,1)
    alb.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM4_sfc.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_SW']
    fin['time'] = np.arange(0,730,1)
    cloudsw.append(fin.mean(dim='lon'))
    #-------------------------------------
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM4_sfc.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_LW']
    fin['time'] = np.arange(0,730,1)
    cloudlw.append(fin.mean(dim='lon'))
    
dTS = xr.concat(dTS,dim='ens')
dTS['ens'] = np.arange(1,101,1)
Tsfc = xr.concat(Tsfc,dim='ens')
Tsfc['ens'] = np.arange(1,101,1)
Tatm = xr.concat(Tatm,dim='ens')
Tatm['ens'] = np.arange(1,101,1)
qlw = xr.concat(qlw,dim='ens')
qlw['ens'] = np.arange(1,101,1)
qsw = xr.concat(qsw,dim='ens')
qsw['ens'] = np.arange(1,101,1)
fsfc = xr.concat(fsfc,dim='ens')
fsfc['ens'] = np.arange(1,101,1)
alb = xr.concat(alb,dim='ens')
alb['ens'] = np.arange(1,101,1)
cloudsw = xr.concat(cloudsw,dim='ens')
cloudsw['ens'] = np.arange(1,101,1)
cloudlw = xr.concat(cloudlw,dim='ens')
cloudlw['ens'] = np.arange(1,101,1)

10
20
30
40
50
60
70
80
90
100


In [6]:
# to convert to temperature contributions, we divide by global average surface temperature feedback
norm = -1 * gw_mean(Tsfc,gw).mean(dim='ens') / gw_mean(dTS,gw).mean(dim='ens')

norm.plot()

<Figure size 432x288 with 1 Axes>

In [40]:
cloud_arc = gw_mean(cloudlw + cloudsw,gw,lat_bound_s=70)/norm
cloud_glo = gw_mean(cloudlw + cloudsw,gw)/norm

q_arc = gw_mean(qsw + qlw,gw,lat_bound_s=70)/norm
q_glo = gw_mean(qsw + qlw,gw)/norm

# fix this to be deviation again?
Tsfc_arc = (gw_mean(Tsfc,gw,lat_bound_s=70)/gw_mean(dTS,gw,lat_bound_s=70) - gw_mean(Tsfc,gw)/gw_mean(dTS,gw)) * gw_mean(dTS,gw,lat_bound_s=70)/norm
Tsfc_glo = (gw_mean(Tsfc,gw)/gw_mean(dTS,gw) - gw_mean(Tsfc,gw)/gw_mean(dTS,gw)) * gw_mean(dTS,gw)/norm

Tatm_arc = gw_mean(Tatm,gw,lat_bound_s=70)/norm
Tatm_glo = gw_mean(Tatm,gw)/norm

alb_arc = gw_mean(alb,gw,lat_bound_s=70)/norm
alb_glo = gw_mean(alb,gw)/norm

fsfc_arc = gw_mean(fsfc,gw,lat_bound_s=70)/norm
fsfc_glo = gw_mean(fsfc,gw)/norm

co2_arc = np.tile(gw_mean((dRFLW_SFC + dRFSW_SFC).mean(dim='lon'),gw,lat_bound_s=70),(2))
co2_glo = np.tile(gw_mean((dRFLW_SFC + dRFSW_SFC).mean(dim='lon'),gw),(2))
co2_tro = np.tile(gw_mean((dRFLW_SFC + dRFSW_SFC).mean(dim='lon'),gw,lat_bound_s=-30,lat_bound_n=30),(2))

